In [59]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
import language_tool_python
##  StratifiedKFold、cross_val_scoreをインポート
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score


In [60]:
test = pd.read_csv('../input/test_essays.csv')
sub = pd.read_csv('../input/sample_submission.csv')
external_train = pd.read_csv("../input/train_essays.csv")
external_train.rename(columns={"generated": "label"}, inplace=True)

train = pd.read_csv("../input/train_v2_drcat_02.csv")
train = pd.concat([train, external_train])

train = train.drop_duplicates(subset=["text"])

train.reset_index(drop=True).sample(frac=1)




,text,label,prompt_name,source,RDizzl3_seven,id,prompt_id
12919,The face on Mars is nothing but a natural land...,0,The Face on Mars,persuade_corpus,True,NaN,NaN
1609,I believe that there are many advanteges of li...,0,Car-free cities,persuade_corpus,True,NaN,NaN
2321,Many people in today's world depend on cars to...,0,Car-free cities,persuade_corpus,True,NaN,NaN
4393,"Dear Generic_Name,\n\nI hope this letter finds...",0,Summer projects,persuade_corpus,False,NaN,NaN
22173,"Dear Principal,\n\nIt has come to my attention...",0,Cell phones at school,persuade_corpus,False,NaN,NaN
...,...,...,...,...,...,...,...
41728,"In the article ""Making Mona Lisa Smile"" the a...",1,Facial action coding system,cohere-command,True,NaN,NaN
17442,Driveless cars are coming. Whether you will bu...,0,Driverless cars,persuade_corpus,True,NaN,NaN
19811,"Dear Senator,\n\nAn Electoral College is a vot...",0,Does the electoral college work?,persuade_corpus,True,NaN,NaN
14748,"Dear principal,\n\nHere is what I think about ...",0,Community service,persuade_corpus,False,NaN,NaN


In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import string

def clean_text(text):
    # HTMLタグの削除
    text = re.sub(r'<.*?>', '', text)
    # 句読点、数字、特殊記号の削除
    text = re.sub(r'[^\w\s]', '', text)
    # 小文字化
    text = text.lower()
    return text

# テキストデータのクリーニング
train['clean_text'] = train['text'].apply(clean_text)
test['clean_text'] = test['text'].apply(clean_text)

In [62]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import MultinomialNB

# TF-IDFベクトル化器を作成
tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

test_x = tfidf.fit_transform(test['clean_text'])
train_x = tfidf.transform(train['clean_text'])

lr = LogisticRegression()
clf = MultinomialNB(alpha=0.02)
sgd_model = SGDClassifier(max_iter=5000, tol=1e-3, loss="modified_huber")   
sgd_model2 = SGDClassifier(max_iter=8000, tol=1e-4, loss="modified_huber", class_weight="balanced") 
sgd_model3 = SGDClassifier(max_iter=10000, tol=5e-4, loss="modified_huber", early_stopping=True)

ensemble = VotingClassifier(
    estimators=[
        ("lr", lr),
        ("mnb", clf),
        ("sgd", sgd_model),
        ("sgd2", sgd_model2),
        ("sgd3", sgd_model3),
    ],
    voting="soft",
)



In [67]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# データをトレーニングセットとテストセットに分割
train_data, test_data, train_labels, test_labels = train_test_split(train['clean_text'], train.label, test_size=0.2, random_state=42)

## tfidfをfit
train_data = tfidf.fit_transform(train_data)
test_data = tfidf.transform(test_data)


# モデルをトレーニング
ensemble.fit(train_data, train_labels)

# テストセットでモデルを評価
predicted_probs = ensemble.predict_proba(test_data)[:, 1]

# AUCを計算
auc = roc_auc_score(test_labels, predicted_probs )
print(f'AUC: {auc}')


AUC: 0.9989262022885921


In [64]:
ensemble.fit(train_x, train.label)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('mnb', MultinomialNB(alpha=0.02)),
                             ('sgd',
                              SGDClassifier(loss='modified_huber',
                                            max_iter=5000)),
                             ('sgd2',
                              SGDClassifier(class_weight='balanced',
                                            loss='modified_huber',
                                            max_iter=8000, tol=0.0001)),
                             ('sgd3',
                              SGDClassifier(early_stopping=True,
                                            loss='modified_huber',
                                            max_iter=10000, tol=0.0005))],
                 voting='soft')

In [65]:
test["generated"] = ensemble.predict_proba(test_x)[:, 1]


In [66]:
#kaggleの提出用ファイルを作成
test[["id", "generated"]].to_csv("submission.csv", index=False)
